In [3]:
import sys
import datetime
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set matplotlib backend for standalone script
plt.switch_backend('TkAgg')  # Use TkAgg for GUI; change to 'Agg' for file output

class IntradayMomentumStrategy(bt.Strategy):
    params = (
        ('num_lots', 1),
        ('lot_size', 75),
    )
    
    def __init__(self):
        self.daily_high = self.datas[0].high
        self.daily_low = self.datas[0].low
        self.daily_volatility = None
        self.upper_boundary = None
        self.lower_boundary = None
        self.order = None
        
        # Data storage for dashboard
        self.data_log = {
            'datetime': [],
            'open': [],
            'close': [],
            'upper_boundary': [],
            'lower_boundary': [],
            'trades': [],  # Format: (datetime, price, type: 'buy'/'sell')
            'portfolio_value': []
        }
        
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.datetime(0)
        print(f'{dt.isoformat()} - {txt}')

    def notify_order(self, order):
        if order.status in [order.Completed]:
            dt = self.datas[0].datetime.datetime(0)
            if order.isbuy():
                self.log(f'BUY EXECUTED at {order.executed.price}')
                self.data_log['trades'].append((dt, order.executed.price, 'buy'))
            elif order.issell():
                self.log(f'SELL EXECUTED at {order.executed.price}')
                self.data_log['trades'].append((dt, order.executed.price, 'sell'))
            self.order = None

    def next(self):
        dt = self.datas[0].datetime.datetime(0)
        open_price = self.datas[0].open[0]
        close_price = self.datas[0].close[0]
        prev_close = self.datas[0].close[-1] if len(self.datas[0]) > 1 else None

        if len(self.datas[0]) < 2:
            return

        # Store data every bar
        self.data_log['datetime'].append(dt)
        self.data_log['open'].append(open_price)
        self.data_log['close'].append(close_price)
        self.data_log['upper_boundary'].append(self.upper_boundary)
        self.data_log['lower_boundary'].append(self.lower_boundary)
        self.data_log['portfolio_value'].append(self.broker.getvalue())

        if dt.hour == 9 and dt.minute == 15:
            self.daily_volatility = self.daily_high[-1] - self.daily_low[-1]
            self.upper_boundary = open_price + self.daily_volatility
            self.lower_boundary = open_price - self.daily_volatility
            self.log(f"Volatility: {self.daily_volatility}, Upper: {self.upper_boundary}, Lower: {self.lower_boundary}")

        if dt.hour == 9 and dt.minute == 30 and not self.position and self.order is None:
            if self.upper_boundary and self.lower_boundary and prev_close:
                if prev_close > self.upper_boundary:
                    self.order = self.buy(size=self.params.num_lots * self.params.lot_size)
                    self.log(f"BUY at {close_price}")
                elif prev_close < self.lower_boundary:
                    self.order = self.sell(size=self.params.num_lots * self.params.lot_size)
                    self.log(f"SELL at {close_price}")

        if dt.hour == 15 and dt.minute == 15 and self.position:
            self.close()
            self.log(f"Position closed at {close_price}")

def create_dashboard(strategy):
    # Convert logged data to DataFrame
    df = pd.DataFrame({
        'datetime': strategy.data_log['datetime'],
        'open': strategy.data_log['open'],
        'close': strategy.data_log['close'],
        'upper_boundary': strategy.data_log['upper_boundary'],
        'lower_boundary': strategy.data_log['lower_boundary'],
        'portfolio_value': strategy.data_log['portfolio_value']
    })
    df.set_index('datetime', inplace=True)

    # Process trades for bar chart (calculate profit/loss)
    trades = pd.DataFrame(strategy.data_log['trades'], columns=['datetime', 'price', 'type'])
    buys = trades[trades['type'] == 'buy']
    sells = trades[trades['type'] == 'sell']

    # Calculate profit/loss per trade pair (assuming buy-then-sell or sell-then-buy)
    trade_profits = []
    if len(buys) > 0 and len(sells) > 0:
        for i in range(min(len(buys), len(sells))):
            buy_price = buys.iloc[i]['price']
            sell_price = sells.iloc[i]['price']
            if buys.iloc[i]['datetime'] < sells.iloc[i]['datetime']:  # Long trade
                profit = (sell_price - buy_price) * strategy.params.lot_size * strategy.params.num_lots
            else:  # Short trade
                profit = (buy_price - sell_price) * strategy.params.lot_size * strategy.params.num_lots
            trade_profits.append({
                'datetime': sells.iloc[i]['datetime'],  # Use exit time
                'profit': profit - (profit * 0.003)  # Adjust for commission
            })
    trade_profits_df = pd.DataFrame(trade_profits)

    # Set seaborn style
    sns.set(style="whitegrid")

    # Create a figure with subplots (3 rows: Line chart, Bar chart, Line chart)
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 12), sharex=True, height_ratios=[3, 1, 1])
    fig.suptitle('Intraday Momentum Strategy Dashboard', fontsize=16)

    # Plot 1: Line Chart - Price and Boundaries
    ax1.plot(df.index, df['close'], label='Close Price', color='blue', linewidth=1.5)
    ax1.plot(df.index, df['upper_boundary'], label='Upper Boundary', color='green', linestyle='--')
    ax1.plot(df.index, df['lower_boundary'], label='Lower Boundary', color='red', linestyle='--')
    ax1.fill_between(df.index, df['upper_boundary'], df['lower_boundary'], color='gray', alpha=0.1)
    ax1.scatter(buys['datetime'], buys['price'], color='green', marker='^', label='Buy', s=100)
    ax1.scatter(sells['datetime'], sells['price'], color='red', marker='v', label='Sell', s=100)
    ax1.set_title('Price Movement and Trades')
    ax1.set_ylabel('Price')
    ax1.legend(loc='upper left')
    ax1.grid(True)

    # Plot 2: Bar Chart - Trade Profits/Losses
    if not trade_profits_df.empty:
        ax2.bar(trade_profits_df['datetime'], trade_profits_df['profit'], 
                color=['green' if p > 0 else 'red' for p in trade_profits_df['profit']], 
                width=0.01)
        ax2.set_title('Profit/Loss per Trade')
        ax2.set_ylabel('Profit (INR)')
        ax2.grid(True)
    else:
        ax2.text(0.5, 0.5, 'No completed trade pairs to display', 
                 horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)

    # Plot 3: Line Chart - Portfolio Value
    ax3.plot(df.index, df['portfolio_value'], label='Portfolio Value', color='purple', linewidth=1.5)
    ax3.set_title('Portfolio Value Over Time')
    ax3.set_ylabel('Value (INR)')
    ax3.set_xlabel('Time')
    ax3.legend(loc='upper left')
    ax3.grid(True)

    # Rotate x-axis labels
    plt.xticks(rotation=45)

    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    try:
        data = bt.feeds.GenericCSVData(
            dataname=r'C:/Users/Ninja/Trade/nifty_ohlc.csv',
            dtformat='%d-%m-%Y %H:%M',
            timeframe=bt.TimeFrame.Minutes,
            compression=15,
            datetime=0,
            open=1,
            high=2,
            low=3,
            close=4,
            volume=5 if 5 < len(pd.read_csv(r'C:/Users/Ninja/Trade/nifty_ohlc.csv', nrows=1).columns) else -1,
            openinterest=-1
        )
        
        cerebro.adddata(data)
        cerebro.addstrategy(IntradayMomentumStrategy)
        cerebro.broker.setcash(1000000.0)
        cerebro.broker.setcommission(commission=0.003)
        
        print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
        strats = cerebro.run()
        strategy = strats[0]
        print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        
        # Call dashboard
        create_dashboard(strategy)
        
    except Exception as e:
        sys.stderr.write(f"Error occurred: {e}\n")
        sys.exit(1)

Starting Portfolio Value: 1000000.00
2015-01-12T09:15:00 - Volatility: 4.600000000000364, Upper: 8295.95, Lower: 8286.75
2015-01-12T09:30:00 - SELL at 8262.35
2015-01-12T09:31:00 - SELL EXECUTED at 8262.15
2015-01-12T15:15:00 - Position closed at 8320.6
2015-01-12T15:16:00 - BUY EXECUTED at 8319.35
2015-01-13T09:15:00 - Volatility: 4.450000000000728, Upper: 8350.6, Lower: 8341.699999999999
2015-01-13T09:30:00 - SELL at 8326.3
2015-01-13T09:31:00 - SELL EXECUTED at 8326.2
2015-01-13T15:15:00 - Position closed at 8303.2
2015-01-13T15:16:00 - BUY EXECUTED at 8303.65
2015-01-14T09:15:00 - Volatility: 4.800000000001091, Upper: 8312.050000000001, Lower: 8302.449999999999
2015-01-14T09:30:00 - SELL at 8302.1
2015-01-14T09:31:00 - SELL EXECUTED at 8301.75
2015-01-14T15:15:00 - Position closed at 8270.2
2015-01-14T15:16:00 - BUY EXECUTED at 8269.6
2015-01-15T09:15:00 - Volatility: 2.9500000000007276, Upper: 8428.150000000001, Lower: 8422.25
2015-01-15T09:30:00 - SELL at 8432.3
2015-01-15T09:31: